In [15]:
import h2o
import h2o.grid
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.11+9-LTS-194, mixed mode)
  Starting server from C:\Users\Gerd\venvs\h2o\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Gerd\AppData\Local\Temp\tmpl6_gxpau
  JVM stdout: C:\Users\Gerd\AppData\Local\Temp\tmpl6_gxpau\h2o_Gerd_started_from_python.out
  JVM stderr: C:\Users\Gerd\AppData\Local\Temp\tmpl6_gxpau\h2o_Gerd_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,3 months and 3 days
H2O_cluster_name:,H2O_from_python_Gerd_d6k8sy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.975 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
url = 'http://h2o-public-test-data.s3.amazonaws.com/smalldata/iris/iris_wheader.csv'
iris = h2o.import_file(url)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
train, test = iris.split_frame([0.8], seed=69)

In [13]:
X = test.columns
X.remove("class")

In [17]:
#
# dataset has 4 input nodes
#
model_autoencoder = H2OAutoEncoderEstimator( hidden=[4], activation='Tanh', epochs=300, model_id='autoencoder',
                                            train_samples_per_iteration=train.nrow, score_interval=0, score_duty_cycle=1.0)

#
# for autoencoders, use None for the target argument
#
%time model_autoencoder.train(X, None, train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%
Wall time: 1.69 s


In [18]:
#
# rmse and mse represent reconstruction error for autoencoders i.e. how similar the 4 output nodes are to the 4 input nodes in this case 
#

model_autoencoder.score_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_mse
0,,2021-06-28 21:34:35,0.089 sec,"0,00000 obs/sec",0.0,0,0.0,0.311757,0.097192
1,,2021-06-28 21:34:35,0.114 sec,4880 obs/sec,1.0,1,122.0,0.223314,0.049869
2,,2021-06-28 21:34:35,0.120 sec,8133 obs/sec,2.0,2,244.0,0.186007,0.034599
3,,2021-06-28 21:34:35,0.126 sec,10764 obs/sec,3.0,3,366.0,0.161461,0.026070
4,,2021-06-28 21:34:35,0.133 sec,12512 obs/sec,4.0,4,488.0,0.142100,0.020192
...,...,...,...,...,...,...,...,...,...
296,,2021-06-28 21:34:36,1.342 sec,34689 obs/sec,296.0,296,36112.0,0.015771,0.000249
297,,2021-06-28 21:34:36,1.345 sec,34740 obs/sec,297.0,297,36234.0,0.013511,0.000183
298,,2021-06-28 21:34:36,1.348 sec,34790 obs/sec,298.0,298,36356.0,0.030780,0.000947
299,,2021-06-28 21:34:36,1.350 sec,34840 obs/sec,299.0,299,36478.0,0.010576,0.000112


In [19]:
h2o.cluster().shutdown()

H2O session _sid_900d closed.
